In [1]:
import torch 
import torch.nn as nn 
import torch.optim as optim
import itertools
from torch.func import  jacrev
from torch import vmap


In [2]:
loaded_dict = torch.load("logs/go1_prilipsnet_v1/Oct26_11-50-42_/model_0.pt")

In [5]:
loaded_dict['model_state_dict'].teacher_adaptation_module

AttributeError: 'collections.OrderedDict' object has no attribute 'teacher_adaptation_module'